# Vertical wall annotations

In [1]:
import geopandas as gpd
import miscelanious.splitRotate as sr
import miscelanious.trsfm_shp as trsfm

In [2]:
shp_file = r"D:\00_Local_scale\vertical_wall_22\02-reprojection\reprojections_annotations_700_labelcor_EPSG32629_290925.shp"
export_file = r"D:\00_Local_scale\vertical_wall_22\05-SplittedRotated\annotations\annotations_290925.csv"

pcd_file = r"D:\00_Local_scale\vertical_wall_22\03-reconstruction\02_mesh_sampling\mesh_sampled_5000_colcor.ply"
plane_dir = r"D:\00_Local_scale\vertical_wall_22\05-SplittedRotated\fitted_planes"

offset = [609000, 5274000]

labels_to_refine = [["SM394 Araeosoma fenestratum",0.05], ["SM92 Actiniaria msp1", 0.0564]]

to_delete = [
"Save for later",
"Incertae sedis",
"Unlisted",
"Unknown species",
"Gadiformes",
"SM1133/SM1143 Madrepora oculata or Desmophyllum pertusum (indistinguishable)",
"SM1133 Madrepora oculata1",
"Fragment",
"Argentiniformes",
"SM440 Trachyscorpia cristulata",
"Actiniaria",
"Ceriantharia",
"Tunicata",
"Antipatharia",
"Asteroidea",
"Leiopathidae",
"Echinoidea",
"Echinidae",
"Pennatulacea",
"SM305 Pseudarchaster",
"Chirostylids",
"SM700 Plexauridae msp6",
"Cidaroida",
"Alcyonacea",
"SM698 Chrysogorgiidae msp.1",
"Crinoidea",
"Primnoidae",
"Whip"
]

label_map = {
"SM235 Bathynectes longispina": "Crabs",
"SM206 Chaceon msp1": "Crabs",
"SM236 Paromola cuvieri": "Crabs",
"True crabs": "Crabs",
"Spider crabs": "Crabs",
"Caryophylliidae": "Solitary",
"Stichopathes sp. (undefined)": "Stichopathes",
"SM130 Stichopathes cf. gravieri": "Stichopathes",
"Crust-like": "Porifera",
"Cup-like": "Porifera",
"Massive": "Porifera",
"SM139 Bathypathes  msp.2": "Schizopathidae",
"SM170 Bathypathes  msp.1": "Schizopathidae",
"SM134 Parantipathes  msp.2": "Schizopathidae",
"SM154 Parantipathes  msp.1": "Schizopathidae",
"SM181 Parantipathes hirondelle": "Schizopathidae",
"SM288 Porania (Porania) pulvillus (orange morph)": "Asteroidea cushion body form",
"SM302 Porania (Porania) pulvillus (purple morph)": "Asteroidea cushion body form",
"Stylasteridae": "Hydrozoa"
}

cut_plane_normal = [-0.527798, 0.847633, -0.0542893]
cut_plane_center = [1045.463257, 205.031754, -809.335571]



In [3]:
annotations_data = gpd.read_file(shp_file)

annotations_data = trsfm.process_labels(annotations_data, label_map, to_delete)

for label, threshold in labels_to_refine:
    annotations_data = trsfm.remove_closest_duplicates(annotations_data, label, threshold)

for label in annotations_data['NAME'].unique():
    annotations_data = trsfm.remove_closest_duplicates(annotations_data, label, 0.02)

annotations_data.set_crs(epsg=32629, inplace=True)

,FID,LAYER,NAME,geometry
0,0,Layer,Hydrozoa,POINT Z (610060.65 5274153.002 -798.901)
1,2,Layer,SM1143 Desmophyllum pertusum,POINT Z (610060.617 5274151.994 -798.185)
2,3,Layer,SM1143 Desmophyllum pertusum,POINT Z (610060.745 5274151.636 -798.135)
3,4,Layer,SM1143 Desmophyllum pertusum,POINT Z (610059.045 5274157.29 -798.685)
4,5,Layer,SM1143 Desmophyllum pertusum,POINT Z (610059.992 5274154.212 -798.488)
...,...,...,...,...
11663,12239,Layer,SM1143 Desmophyllum pertusum,POINT Z (610012.094 5274217.505 -794.478)
11664,12240,Layer,SM1143 Desmophyllum pertusum,POINT Z (610011.014 5274217.7 -794.477)
11665,12241,Layer,SM394 Araeosoma fenestratum,POINT Z (610015.357 5274220.109 -809.343)
11666,12242,Layer,SM1143 Desmophyllum pertusum,POINT Z (610001.676 5274226.827 -806.787)


Compute plane equations

In [4]:
#faces = sr.compute_faces(pcd_file, plane_dir, cut_plane_normal, cut_plane_center)

Load plane equations

In [5]:
faces = sr.load_faces(plane_dir)

In [6]:
annotations_pcd = trsfm.gpd2pv(annotations_data, offset=offset)
annotations_pcd['Classification'] = annotations_data.NAME

ann_split = sr.split_rotate_annotations(annotations_pcd, cut_plane_normal, cut_plane_center, faces)

for i, ann in ann_split.items():
    export_path = export_file.replace(".csv", f"_{i}.csv")
    ann.to_csv(export_path, index = False)
    print(f"Exported {export_path}")

Clipping with Function: 100%|██████████[00:00<00:00]
Clipping with Function: 100%|██████████[00:00<00:00]

Exported D:\00_Local_scale\vertical_wall_22\05-SplittedRotated\annotations\annotations_290925_left.csv
Exported D:\00_Local_scale\vertical_wall_22\05-SplittedRotated\annotations\annotations_290925_right.csv
